### Book Store Record Application

This database application stores book records with title, author,
year and ISBN information. Each record in the database is unique based on all four fields (i.e. No duplication of records is allowed. However, record with the same title, author and ISBN but different year is allowed. Record adding and updating checks for record dupcation first before record is added or updated. The Graphical User Interface provides users multiple functions including:
   - Displaying all records
   - Adding records with search for record duplicates first
   - Deleting selected records
   - Updating selected records with search for record duplicates first
   - Searching records with any one or more of the four record fields
   - Status bar displaying status of adding, deleting and updating records
<br>   
<img src="bookstore_app_snap.png" width = 70%>

In [1]:
from tkinter import *
import sqlite3

In [2]:
db = "mybooks_oop.db"

In [3]:
class Books_db():
    
    #Create/Connect to database file
    def __init__(self, db):
        self.__db_file = db  #Database
        self.__conn=sqlite3.connect(self.__db_file)
        self.__cur=self.__conn.cursor()
        self.__cur.execute("CREATE TABLE IF NOT EXISTS book_records (title TEXT, author TEXT, year INTEGER, isbn TEXT)")
        self.__conn.commit()
        
    def __del__(self):
        self.__conn.close()

    #Return database file name
    def get_db(self):
        return self.__db_file
        
    #Insert record into database
    def insert(self, title, author, year, isbn):
        self.__cur.execute("INSERT INTO book_records VALUES(?,?,?,?)",(title, author, int(year), isbn))
        self.__conn.commit()
        
    #Remove record from database
    def delete(self, title, author, year, isbn):
        self.__cur.execute("DELETE FROM book_records WHERE title=? AND author=? AND year=? AND isbn=?",(title, author, year, isbn))
        self.__conn.commit()

    #Update record from database
    def update(self, title, author, year, isbn, current_title, current_author, current_year, current_isbn):
        self.__cur.execute("UPDATE book_records SET title=?, author=?, year=?, isbn=? WHERE title=? AND author=? AND year=? AND isbn=?",(title, author, year, isbn, current_title, current_author, current_year, current_isbn))
        self.__conn.commit()

    #Retrieve all records from database
    def view(self):
        self.__cur.execute("SELECT * FROM book_records")
        rows=self.__cur.fetchall()
        return rows
    
    #Search record from database
    def search(self, title, author, year, isbn):
        command = "SELECT * FROM book_records"
        col = ["title", "author", "year", "isbn"]
        parameter = [title, author, year, isbn]
        command_p = []
        first_para = True

        if not all (field=="" for field in parameter):   #SAME AS if not all([title=="", author=="", year=="", isbn==""]):
            command = command + " WHERE"
            for idx, value in enumerate (parameter):
                if value!="":
                    if first_para == False: 
                        command = command + " AND"
                    command = command + " " + col[idx] + "=?"
                    command_p.append(value)
                    first_para = False                

        self.__cur.execute(command,command_p)
        rows=self.__cur.fetchall()
        return rows

In [7]:
class Gui():
    def __init__(self, root, db_file):
        self.books_db = Books_db(db_file)
        self.root = root
        self.root.title("Book Record Database")

        self.l1 = Label(self.root, text="Title")  #Title label
        self.l1.grid(row=0, column=0)

        self.l2 = Label(self.root, text="Author") #Author Label
        self.l2.grid(row=0, column=2)

        self.l3 = Label(self.root, text="Year")   #Year Label
        self.l3.grid(row=1, column=0)

        self.l4 = Label(self.root, text="ISBN")   #ISBN label
        self.l4.grid(row=1, column=2)

        self.title = StringVar()
        self.e1 = Entry(self.root, textvariable=self.title)    #Title entry box
        self.e1.grid(row=0, column=1)

        self.author = StringVar()
        self.e2 = Entry(self.root, textvariable=self.author)   #Author entry box
        self.e2.grid(row=0, column=3)

        self.year = StringVar()
        self.e3 = Entry(self.root, textvariable=self.year)     #Year entry box
        self.e3.grid(row=1, column=1)

        self.isbn = StringVar()
        self.e4 = Entry(self.root, textvariable=self.isbn)     #ISBN entry box
        self.e4.grid(row=1, column=3)

        self.scrollbar = Scrollbar(self.root)             #Scroll bar for record listbox
        self.scrollbar.grid(row=2, column=4, rowspan=4, sticky='ns')

        #Record listbox
        self.t1 = Listbox(self.root, height=7, width=55, yscrollcommand = self.scrollbar.set)
        self.t1.grid(row=2, column=0, rowspan=4, columnspan=4)
        self.scrollbar.config(command = self.t1.yview) 
        self.t1.bind("<<ListboxSelect>>", self.callback)

        #View all record button
        self.b1 = Button(self.root, text="View All", width=12, command=self.view_records)
        self.b1.grid(row=0, column=5)

        #Search record button
        self.b2 = Button(self.root, text="Search Entry", width=12, command=self.search_records)
        self.b2.grid(row=1, column=5)

        #Add entry button
        self.b3 = Button(self.root, text="Add Entry", width=12, command=self.add_records)
        self.b3.grid(row=2, column=5)

        #Update record button
        self.b4 = Button(self.root, text="Update Selected", width=12, command=self.update_records)
        self.b4.grid(row=3, column=5)

        #Delete record button
        self.b5 = Button(self.root, text="Delete Selected", width=12, command=self.delete_records)
        self.b5.grid(row=4, column=5)

        #Close application button
        self.b6 = Button(self.root, text="Close", width=12, command=self.root.destroy)
        self.b6.grid(row=5, column=5)

        self.status = StringVar()
        self.l5 = Label(self.root, textvariable=self.status)    #Status Label
        self.l5.grid(row=6, column=0, columnspan=4)

    #View all records in database and display it on the GUI
    def view_records(self):
        for idx in reversed(range(self.t1.size())):        #Same as t1.delete(0.END)
            self.t1.delete(idx)                             
        for idx, book in enumerate (self.books_db.view(), start=1):
            self.t1.insert(idx, " [" + str(idx) + "] " + book[0] + ", " + book[1] + ", " + str(book[2]) + ", " + book[3] + "\n")
        self.status.set("")

    #Add record into the datbase with info entered from the GUI.
    #Record can only be added if it does not exist in database and 
    #all record fields provided are non-empty.
    def add_records(self):
        title_str = str(self.title.get())
        author_str = str(self.author.get())
        year_str = str(self.year.get())
        isbn_str = str(self.isbn.get())

        if not any([title_str=="", author_str=="", isbn_str=="", year_str==""]):
            current_record = self.books_db.search(title_str, author_str, year_str, isbn_str)
            if len(current_record) == 0:
                self.books_db.insert(title_str, author_str, year_str, isbn_str)
                self.view_records()
                self.status.set("Record added successfully.")
            else:
                self.status.set("Record already exists.")
        else:
            self.status.set("One or more field is mssing. Unable to add record.")

    #Update record selected in the GUI. Name is used to retreive record from
    #database and enterd author, year and ISBN info are updated in the database.
    def update_records(self):
        title_str = str(self.title.get())
        author_str = str(self.author.get())
        year_str = str(self.year.get())
        isbn_str = str(self.isbn.get())

        if not any([title_str=="", author_str=="", isbn_str=="", year_str==""]):
            new_record = self.books_db.search(title_str, author_str, year_str, isbn_str)
            if len(new_record) == 0:
                idx = self.t1.curselection()[0]  #Index of selection
                current_record = self.t1.get(idx)
                current_record = current_record.strip('\n')
                current_record = current_record.split("] ", 1)
                current_record = current_record[1].split(", ", 3)

                self.books_db.update(title_str, author_str, year_str, isbn_str, 
                       current_record[0], current_record[1], 
                       current_record[2], current_record[3])
                #all_record = True
                self.view_records()
                self.status.set("Record updated successfully.")
            else:
                self.status.set("Record exists already. Record is not updated.")
        else:
            self.status.set("One or more field is mssing. Unable to add record.")

    #Remove record selected in the GUI from the database. 
    def delete_records(self):
        title_str = str(self.title.get())
        author_str = str(self.author.get())
        year_str = str(self.year.get())
        isbn_str = str(self.isbn.get())

        if not any([title_str=="", author_str=="", isbn_str=="", year_str==""]):
            new_record = self.books_db.search(title_str, author_str, year_str, isbn_str)
            if len(new_record) != 0:
                self.books_db.delete(title_str, author_str, year_str, isbn_str)
                self.view_records()
                self.status.set("Record deleted successfully.")
            else:
                self.status.set("Record does not exist. No record is deleted.")
        else:
            self.status.set("One or more field is mssing. Unable to add record.")

    #Search record from the database that matches with title, author, 
    #year and ISBN info entered in the GUI. Any one or more field 
    #can be used for the search.
    def search_records(self):
        title_str = str(self.title.get())
        author_str = str(self.author.get())
        year_str = str(self.year.get())
        isbn_str = str(self.isbn.get())
        rows = self.books_db.search(title_str, author_str, year_str, isbn_str)

        for idx in reversed(range(self.t1.size())):
            self.t1.delete(idx)
        for idx, book in enumerate (rows, start=1):
            self.t1.insert(idx, " [" + str(idx) + "] " + book[0] + ", " + book[1] + ", " + str(book[2]) + ", " + book[3] + "\n")
        self.status.set("")

    #Callback function to display record details user selected in the listbox.
    def callback(self, event):
        selection = event.widget.curselection()
        if selection:
            idx = selection[0]
            book = self.t1.get(idx)
            book = book.strip('\n')
            book = book.split("] ", 1)
            book = book[1].split(", ", 3)

            self.title.set(book[0])
            self.author.set(book[1])
            self.year.set(book[2])
            self.isbn.set(book[3])

In [8]:
#GUI Execution
root = Tk()
gui = Gui(root, db)
gui.root.mainloop()

In [6]:
#Sample table and records for testing

#create_table()
#insert("Pete the Cat: Super Pete", "James Dean", 2020, "978-0062868503")
#insert("Pete the Cat and His Four Groovy Buttons", "Eric Litwin", 2012, "0062110586")
#insert("Pete the Cat: The Wheels on the Bus", "James Dean", 2015, "978-0062358523")
#insert("Test1", "JTest", 2000, "84750373")
#insert("Test2", "JTest", 2000, "84940374")
#insert("Test3", "JTest", 1999, "12441222")

#for row in view():
#    print(row)